# Data

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

## Dataset

### Rossmann Store Sales
Forcaste the sales column. (number of Sales) Regression.


It is not imbalanced but there are only 9 columns. 

In [3]:
train = pd.read_csv("/Users/sepideghorbanian/Documents/Semester_5/Research_Project/Data/rossmann-store-sales/train.csv")
store = pd.read_csv("/Users/sepideghorbanian/Documents/Semester_5/Research_Project/Data/rossmann-store-sales/store.csv")
data = train.merge(store, on='Store', how='inner')
data

/var/folders/m_/m50wq84x1gv7lmgz099xvzd80000gn/T/ipykernel_52007/2911457916.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv("/Users/sepideghorbanian/Documents/Semester_5/Research_Project/Data/rossmann-store-sales/train.csv")


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1017205,1112,2,2013-01-01,0,0,0,0,a,1,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1017206,1113,2,2013-01-01,0,0,0,0,a,1,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1017207,1114,2,2013-01-01,0,0,0,0,a,1,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 18 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Store                      1017209 non-null  int64  
 1   DayOfWeek                  1017209 non-null  int64  
 2   Date                       1017209 non-null  object 
 3   Sales                      1017209 non-null  int64  
 4   Customers                  1017209 non-null  int64  
 5   Open                       1017209 non-null  int64  
 6   Promo                      1017209 non-null  int64  
 7   StateHoliday               1017209 non-null  object 
 8   SchoolHoliday              1017209 non-null  int64  
 9   StoreType                  1017209 non-null  object 
 10  Assortment                 1017209 non-null  object 
 11  CompetitionDistance        1014567 non-null  float64
 12  CompetitionOpenSinceMonth  693861 non-null   float64
 13  CompetitionO

In [5]:
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['WeekOfYear'] = data['Date'].dt.isocalendar().week

# Drop columns we don't want
data = data.drop(columns=['Date', 'Customers']) # Customers is correlated with Sales, try both with and without it

## Startified

### Split the data into 5 datasets startified.

The problem with regression is that because it has continues values for the label it is not possible to easily do the startification (like in classification). 
The trick is to first split it into different bins (pandas qcut: Quantile-based discretization function , first 10% of the values, second 10% of the values and so on). Then use those bins for startification. Basically it makes sure to take the same amount of records from each bin. Use Kfold for making the 5 datasets and then train_test_split for splitiing each of those datasets.


In [ ]:
y = data["Sales"]
X = data.drop("Sales" , axis = 1)

In [ ]:
# Quantile bins for stratification, turn them to categories based on their index. (Kfold needs y to be category)
y_bins = pd.qcut(y, q=10, duplicates='drop')
y_bins = y_bins.cat.codes

In [ ]:
# Create 5 stratified subsets
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

splits = []
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y_bins)):
    X_subset = X.iloc[test_idx]
    y_subset = y.iloc[test_idx]
    splits.append((X_subset, y_subset))
    print(f"Subset {fold+1}: {X_subset.shape}")

In [ ]:
datasets = []  # Store (X_train, X_test, y_train, y_test) for each subset

for i, (X_sub, y_sub) in enumerate(splits):
    # Re-bin within subset for stratification
    y_bins_sub = pd.qcut(y_sub, q=10, duplicates='drop')
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_sub, y_sub, test_size=0.2, stratify=y_bins_sub, random_state=42
    )
    
    datasets.append((X_train, X_test, y_train, y_test))
    print(f"Subset {i+1}: Train={X_train.shape}, Test={X_test.shape}")

In [ ]:
X_train_1, X_test_1, y_train_1, y_test_1 = datasets[0]
X_train_2, X_test_2, y_train_2, y_test_2 = datasets[1]
X_train_3, X_test_3, y_train_3, y_test_3 = datasets[2]
X_train_4, X_test_4, y_train_4, y_test_4 = datasets[3]
X_train_5, X_test_5, y_train_5, y_test_5 = datasets[4]

In [ ]:
X_main_train = pd.concat([X_train_1, X_train_2, X_train_3, X_train_4 , X_train_5 ]).reset_index(drop=True)
y_main_train = pd.concat([y_train_1, y_train_2, y_train_3, y_train_4, y_train_5 ]).reset_index(drop=True)
X_main_test = pd.concat([X_test_1, X_test_2, X_test_3, X_test_4 , X_test_5]).reset_index(drop=True)
y_main_test = pd.concat([y_test_1, y_test_2, y_test_3, y_test_4, y_test_5]).reset_index(drop=True)

In [ ]:
main_train_df = X_main_train.copy()
main_train_df['Sales'] = y_main_train
main_train_df

In [ ]:
main_test_df = X_main_test.copy()
main_test_df['Sales'] = y_main_test
main_test_df

In [ ]:
main_train_df.to_csv("main_train_df.csv")
main_test_df.to_csv("main_test_df.csv")